# Алгоритм иерархической кластеризации с алгоритмом объединения "single-link clustering" (SLINK)

Алгоритм начинает работу, считая каждую точку отдельным кластером. На каждой итерации он находит два максимально близких кластера (с минимальным расстоянием между ближайшими элементами) и сливает их в один. Это продолжается, пока все точки не окажутся в одном кластере.

Сначала алгоритм рассчитывает матрицу расстояний $ D $ (для удобства диагональ заполняется бесконечностями). Также для каждой точки в двух дополнительных массивах хранится индекс ближайшего соседа и расстояние до него.

Это занимает $O(n^2)$ времени.

На каждой итерации: 
 - находится минимум в массиве расстояний до ближайшего соседа и две соответствующие точки $i$ и $j$
 - в массиве расстояний до ближайшего соседа на j-том месте ставится бесконечность, что соответствует удалению $j$ из рассмотрения
 - пересчитывается i-тая строчка матрицы расстояний: $D(i,x) = min( D(i,x), D(j,x) )$. Теперь i-тая строчка соответствует кластеру $ij$
 - для кластера $ij$ находится новый ближайший сосед и расстояние до него и записывается в i-тую ячейку соответствующих массивов
 - если для какой-то точки ближайшим соседом была $j$, то для нее ближайшим соседом становится $i$

$O(n)$ итераций, на каждой из них тратится $O(n)$ времени на поиск минимума в массивах и обновление строки в матрице.<br>
Итого сложность по времени $O(n^2)$

In [2]:
import numpy as np

def proximity_matrix(x):
    max_dist = max(x) - min(x) + 1
    
    proximity = np.eye(len(x))
    for i,a in enumerate(x):
        for j,b in enumerate(x):
            if i == j:
                proximity[i,j] = max_dist
            else:
                proximity[i,j] = abs(a - b)
    return proximity

def SLINKcluster(x):
    max_dist = max(x) - min(x) + 1
    clusters = [[i] for i in x]

    proximity = proximity_matrix(x)
    dist_list = np.min(proximity, axis=0)
    nearest_list = np.argmin(proximity, axis=0) 
    
    for step in range(len(x)-1):
        i = np.argmin(dist_list)
        j = nearest_list[i]

        dist_list[j] = max_dist
        
        for k,a in enumerate(x):
            if k == i or k == j:
                d = max_dist
            else:
                d = min(proximity[i,k], proximity[j,k])
            proximity[i,k] = d
            proximity[k,j] = max_dist 
        
        dist_list[i] = min(proximity[i])
        nearest_list[i] = np.argmin(proximity[i])

        for k,n in enumerate(nearest_list):
            if n == j: 
                nearest_list[k] = i

        clusters[i] = [clusters[i], clusters[j]]
    return clusters[i]

In [3]:
import random
x = [random.randrange(100) for i in range(10)]
# x = [float(a) for a in input().split()]

print(SLINKcluster(x))

[[[[[[55], [54]], [[42], [46]]], [64]], [[[28], [[16], [17]]], [4]]], [93]]


Недостаток этого метода в том, что он производит "длинные" кластеры, в которых расстояние между соседями мало, но расстояние между элементами на двух противоположных концах кластера велико.

Также метод имеет проблему "цепных кластеров", в которых к кластеру каждый раз добавляется одиночный элемент.<br>
Пример: на точках $x = [1, 2, 4, 6, 8]$ получатся кластеры $[[[[1,2],4],6],8]$